In [1]:
# Note: in practice, one should put all the imports at the top of a Python script. 
# But for educational purposes here, I put the import immediately before usage. 
# This Python block is just to set up some plotting stuff.

# Imports for plotting
import openpathsampling as paths
import numpy as np

In [2]:
from openpathsampling.toy_dynamics.toy_engine import ToyEngine
from openpathsampling.toy_dynamics.toy_integrators import LangevinBAOABIntegrator
from openpathsampling.toy_dynamics.toy_pes import Gaussian, OuterWalls

#import logging.config
#logging.config.fileConfig("logging.conf", disable_existing_loggers=False)

### Set up the toy system
This includes the potential energy surface (PES), and an initial configuration

In [3]:
# Toy_PES supports adding/subtracting various PESs. The OuterWalls PES type gives an x^6+y^6 boundary to the system
pes = (OuterWalls([1.0, 1.0], [0.0, 0.0]) + 
       Gaussian(2.0, [1.0, 4.0], [0.0,1.0]) + 
       Gaussian(2.0, [25.0, 1.0], [0.0, -1.0])
      )
topology=paths.ToyTopology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)
template = paths.Snapshot(
    coordinates=np.array([[-0.5, 0.0]]), 
    velocities=np.array([[0.0,0.0]]),
    potential_energy = 0.0, # Important to set energies dimensionless otherwise kcal/mol is default
    kinetic_energy = 0.0,
    topology=topology
)

### Setup engine

In [4]:
integ = LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=1.0)
options={
         'integ' : integ,
         'n_frames_max' : 5000,
         'nsteps_per_frame' : 10}
sim = ToyEngine(
        options=options,
        template=template
)

sim.current_snapshot = template

### Storage setup

In [5]:
storage = paths.storage.Storage('toy_dynamics_storage.nc', mode='w', template=template)

In [6]:
from openpathsampling.orderparameter import OP_Function
from openpathsampling.volume import VolumeFactory as vf
from openpathsampling.volume import LambdaVolume
from openpathsampling.trajectory import Trajectory

# Note that we select from the first atom coordinate one, although
# there is only one atom. This is to make it consistant with
# multiple atom simulations!
try:
    op_line = OP_Function(name="xval", 
                          fcn=lambda snap : snap.coordinates[0,0])
except ValueError:
    print "xval already made"
# this defines a set of LambdaVolumes for which the minimum is -1.0 and the maximum is given in the array
line_volume_set = vf.LambdaVolumeSet(op_line, minvals=-1.0, maxvals=[-0.30, -0.25, -0.20, -0.13, -0.05, 0.0])
stateA = LambdaVolume(op_line, -1.0, -0.3)
stateB = LambdaVolume(op_line, 0.3, 1.0)

In [7]:
from openpathsampling.ensemble import EnsembleFactory as ef
from openpathsampling.ensemble import LengthEnsemble, AllInEnsemble, AllOutEnsemble, SequentialEnsemble, OptionalEnsemble, SingleFrameEnsemble

interface0 = line_volume_set[0]
interface_ensemble_set = ef.TISEnsembleSet(stateA, stateB, line_volume_set)
first_traj_ensemble = SequentialEnsemble([
    OptionalEnsemble(AllOutEnsemble(stateA)),
    AllInEnsemble(stateA),
    OptionalEnsemble(AllOutEnsemble(stateA) & AllInEnsemble(interface0)),
    OptionalEnsemble(AllInEnsemble(interface0)),
    AllOutEnsemble(interface0),
    OptionalEnsemble(AllOutEnsemble(stateA)),
    SingleFrameEnsemble(AllInEnsemble(stateA))
])
first_traj_ensemble.name = 'first trajectory ensemble'

In [8]:
sim.initialized = True
first_traj = sim.generate(sim.current_snapshot, [first_traj_ensemble.can_append])

In [9]:
subtraj = interface_ensemble_set[0].split(first_traj)[0]

### Saving some stuff

In [10]:
storage.save(first_traj);
storage.save(subtraj);
storage.save(first_traj_ensemble);

Don't forget to sync once in a while!

In [11]:
storage.sync() # necessary to make sure all changes are stored to disk!

In [12]:
from openpathsampling.pathmover import PathMoverFactory as mf
from openpathsampling.shooting import UniformSelector

# might be nice to get rid of these two lines somehow
from openpathsampling.pathmover import PathMover
PathMover.engine = sim

shooting_movers = mf.OneWayShootingSet(UniformSelector(), interface_ensemble_set)

In [13]:
from openpathsampling.calculation import Bootstrapping

# TODO: add storage
bootstrap = Bootstrapping(storage=None,
                          engine=sim,
                          ensembles=interface_ensemble_set,
                          movers=shooting_movers,
                          trajectory=subtraj)

In [14]:
bootstrap.run(300)

In [15]:
from openpathsampling.pathmover import PathReversalSet
reversal_movers = PathReversalSet(interface_ensemble_set)

In [16]:
from openpathsampling.pathmover import RandomChoiceMover
shooting_chooser = RandomChoiceMover(shooting_movers)
reversal_chooser = RandomChoiceMover(reversal_movers)
equilibration_movers = RandomChoiceMover([shooting_chooser, reversal_chooser])

In [17]:
from openpathsampling.calculation import PathSampling
equilibration = PathSampling(storage=None, engine=sim, root_mover=equilibration_movers, globalstate=bootstrap.globalstate)

In [18]:
# does not work with the current configuration of creating an initial ensemble
# bootstrapping uses ONLY 1 replica, but several ensembles, this is also
# in the tests. I assume here we want to use a set of several replica so we
# need to rewrite this in either way. But @David know here more

#equilibration.run(200)

In [19]:
from openpathsampling.pathmover import NeighborEnsembleReplicaExchange
repex_movers = NeighborEnsembleReplicaExchange(interface_ensemble_set)
repex_chooser = RandomChoiceMover(repex_movers)

In [20]:
retis_root_mover = RandomChoiceMover([shooting_chooser, reversal_chooser, repex_chooser])
production = PathSampling(storage=storage, engine=sim, root_mover=retis_root_mover, globalstate=equilibration.globalstate)

In [21]:
#production.run(500)